In [40]:
import pandas as pd
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests 
from scipy import stats
from collections import defaultdict

In [3]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [5]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [6]:
description_columns = ['Patient_id', 'Diagnosis']
gene_columns = [x for x in data.columns if x not in description_columns]

In [43]:
def fold_change(t, c):
    if t > c:
        return t / c

    if c > t:
        return -c / t

    return 1.0

In [56]:
p_values = defaultdict(list)

In [57]:
def create_p_values(c_name, t_name):
    C = data[data.Diagnosis == c_name]
    T = data[data.Diagnosis == t_name]

    for name in gene_columns:
        cur_c = C[[name]]
        cur_t = T[[name]]
        res = stats.ttest_ind(cur_c, cur_t,  equal_var=False)
        p_values[(c_name, t_name)].append(res.pvalue[0])

In [58]:
c_name = 'normal'
t_name = 'early neoplasia'
create_p_values(c_name, t_name)


In [59]:
c_name = 'early neoplasia'
t_name = 'cancer'
create_p_values(c_name, t_name)

In [60]:
alpha = 0.05
for (c_name, t_name), values in p_values.items():
    count = 0
    for p in values:
        if p < alpha:
            count += 1
    print(c_name, '/', t_name, ':', count)

    with open(f'gene_high_throughput_sequencing_1_{c_name.replace(" ", "_")}_{t_name.replace(" ", "_")}.txt', 'w') as fp:
        fp.write(str(count))

normal / early neoplasia : 1575
early neoplasia / cancer : 3490


In [64]:
alpha = 0.05
alpha_corrected = alpha / 2
for (c_name, t_name), values in p_values.items():
    reject, p_corrected, a1, a2 = multipletests(values, alpha=alpha, method='holm')

    count = 0
    for idx, p in enumerate(p_corrected):
        if p >= alpha_corrected:
            continue
        col_name = gene_columns[idx]
        c = np.mean(data[data.Diagnosis == c_name][[col_name]].values)
        t = np.mean(data[data.Diagnosis == t_name][[col_name]].values)
        fc = fold_change(t, c)
        if abs(fc) > 1.5:
            count += 1
    print(c_name, '/', t_name, ':', count)

    with open(f'gene_high_throughput_sequencing_2_{c_name.replace(" ", "_")}_{t_name.replace(" ", "_")}.txt', 'w') as fp:
        fp.write(str(count))

normal / early neoplasia : 2
early neoplasia / cancer : 77


In [66]:
alpha = 0.05
alpha_corrected = alpha / 2
for (c_name, t_name), values in p_values.items():
    reject, p_corrected, a1, a2 = multipletests(values, alpha=alpha, method='fdr_bh')

    count = 0
    for idx, p in enumerate(p_corrected):
        if p >= alpha_corrected:
            continue
        col_name = gene_columns[idx]
        c = np.mean(data[data.Diagnosis == c_name][[col_name]].values)
        t = np.mean(data[data.Diagnosis == t_name][[col_name]].values)
        fc = fold_change(t, c)
        if abs(fc) > 1.5:
            count += 1
    print(c_name, '/', t_name, ':', count)

    with open(f'gene_high_throughput_sequencing_3_{c_name.replace(" ", "_")}_{t_name.replace(" ", "_")}.txt', 'w') as fp:
        fp.write(str(count))

normal / early neoplasia : 4
early neoplasia / cancer : 524
